In [1]:
! pip install keybert

     |████████████████████████████████| 79 kB 3.4 MB/s 
     |████████████████████████████████| 224 kB 24.3 MB/s 
     |████████████████████████████████| 51 kB 6.8 MB/s 
     |████████████████████████████████| 3.8 MB 44.2 MB/s 
     |████████████████████████████████| 1.2 MB 24.4 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 895 kB 43.8 MB/s 
     |████████████████████████████████| 6.5 MB 29.8 MB/s 
     |████████████████████████████████| 596 kB 39.3 MB/s 
  Created wheel for keybert: filename=keybert-0.5.0-py3-none-any.whl size=20491 sha256=7f2f7655945d15e5a1eeec74c68ce8348888a278a733c53b8d8c212ba01be880
  Stored in directory: /root/.cache/pip/wheels/99/1f/3f/590d2997adbb2d0e1f82e8ee05d42d6910e92c3ed283015ff8
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=c25a7b9dbfc16b83e69bd7fe228c7e06e61840ac55749ed504e89fa022cc006c
  Stored in directory: /root/.cache/pip/whee

In [2]:
! pip install sentence_transformers

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import nltk
from google.colab import drive
import re
import en_core_web_sm
from collections import defaultdict
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer, util

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
nlp = spacy.load("en_core_web_sm")

In [50]:
symmetric_model = SentenceTransformer('all-mpnet-base-v2')
asymmetric_model =  SentenceTransformer('msmarco-MiniLM-L-6-v3')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
#Reading the data 
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/raw_paragraphs_0803_to_1203.csv")

In [9]:
#industry = Fashion, jewelery, country = UK
data.head()

,Unnamed: 0,id,resourceLink,tags,_ts,paragraph
0,0,4cc1e994-51c4-5aa9-9a06-c29a2de5174c.json,https://www.responsiblejewellery.com/wp-conten...,"{'external': [], 'benefit': 'energy saving', '...",1646934908,chain that promotes trust in the global jewell...
1,1,4cc1e994-51c4-5aa9-9a06-c29a2de5174c.json,https://www.responsiblejewellery.com/wp-conten...,"{'external': [], 'benefit': 'energy saving', '...",1646934908,Jewellery has always played a special role in ...
2,2,4cc1e994-51c4-5aa9-9a06-c29a2de5174c.json,https://www.responsiblejewellery.com/wp-conten...,"{'external': [], 'benefit': 'energy saving', '...",1646934908,It was founded in 2005 by 14 member organisati...
3,3,4cc1e994-51c4-5aa9-9a06-c29a2de5174c.json,https://www.responsiblejewellery.com/wp-conten...,"{'external': [], 'benefit': 'energy saving', '...",1646934908,"Today our membership has grown to more than 1,..."
4,4,4cc1e994-51c4-5aa9-9a06-c29a2de5174c.json,https://www.responsiblejewellery.com/wp-conten...,"{'external': [], 'benefit': 'energy saving', '...",1646934908,Our actions support the industry to be even mo...


In [72]:
data.columns

Index(['Unnamed: 0', 'id', 'resourceLink', 'tags', '_ts', 'paragraph'], dtype='object')

In [10]:
data.shape

(175604, 6)

In [12]:
data.isnull().sum()

Unnamed: 0      0
id              0
resourceLink    0
tags            0
_ts             0
paragraph       0
dtype: int64

In [13]:
data.duplicated().sum()

0

In [16]:
# def fashion_preprocessing(text):
#   text= re.sub(r'\n',"",text)
#   text= re.sub(r'http://www.\w+.org/','',text)
#   text= re.sub(r'Figure\s\d:','', text)
#   text= re.sub(r'https://\w+.\w+/\d+.\d+/\w\d+\W\w+','',text)
#   text = re.sub('https?://\S+|www\.\S+', '', text)
#   text= re.sub(r'\Wwww.\w+\W\w+\W','',text)
#   text= re.sub(r'www.\w+','',text)
#   regrex_pattern = re.compile(pattern = "["
#         u"\U0001F600-\U0001F64F"  # emoticons
#         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#         u"\U0001F680-\U0001F6FF"  # transport & map symbols
#         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#         u"\U00002500-\U00002BEF"  # chinese char
#         u"\U00002702-\U000027B0"
#         u"\U00002702-\U000027B0"
#         u"\U000024C2-\U0001F251"
#         u"\U0001f926-\U0001f937"
#         u"\U00010000-\U0010ffff"
#         u"\u2640-\u2642" 
#         u"\u2600-\u2B55"
#         u"\u200d"
#         u"\u23cf"
#         u"\u23e9"
#         u"\u231a"
#         u"\ufe0f"  # dingbats
#         u"\u3030"  # flags (iOS)
#                            "]+", flags = re.UNICODE)
#   text= regrex_pattern.sub('', text)
#   tokens = [token for token in nlp(text) if not (token.is_stop or token.like_num or token.is_punct or token.is_space or len(token)==1)]
#   text= " ".join(map(str,tokens))
#   return text

In [71]:
class para_relevancy(object):

  def __init__(self, data):
    """
    data ----> DataFrame along with the columns
               
    """
    self.__dataframe = data
    self.__content = self.__dataframe['paragraph'].tolist()
    #preprocessing the paras
    self.preprocessed = list(map(self.__preprocessor, self.__content))
    print("\n Preprocessing has been completed")

  #preprocessing function
  def __preprocessor(self,text):
    text= re.sub(r'\n',"",text)
    text= re.sub(r'http://www.\w+.org/','',text)
    text= re.sub(r'Figure\s\d:','', text)
    text= re.sub(r'https://\w+.\w+/\d+.\d+/\w\d+\W\w+','',text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text= re.sub(r'\Wwww.\w+\W\w+\W','',text)
    text= re.sub(r'www.\w+','',text)
    regrex_pattern = re.compile(pattern = "["
          u"\U0001F600-\U0001F64F"  # emoticons
          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
          u"\U0001F680-\U0001F6FF"  # transport & map symbols
          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
          u"\U00002500-\U00002BEF"  # chinese char
          u"\U00002702-\U000027B0"
          u"\U00002702-\U000027B0"
          u"\U000024C2-\U0001F251"
          u"\U0001f926-\U0001f937"
          u"\U00010000-\U0010ffff"
          u"\u2640-\u2642" 
          u"\u2600-\u2B55"
          u"\u200d"
          u"\u23cf"
          u"\u23e9"
          u"\u231a"
          u"\ufe0f"  # dingbats
          u"\u3030"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    text= regrex_pattern.sub('', text)
    tokens = [token for token in nlp(text) if not (token.is_stop or token.like_num or token.is_punct or token.is_space or len(token)==1)]
    text= " ".join(map(str,tokens))
    return text

  def unigram_key_word_extraction(self, save = False):
      """
      This method extracts the most significant key word from the given paras.
      Dataframe will be saved with the keywords and probabilites if save is switch to TRUE
      
      """

      key_model = KeyBERT()
      self.__unigram_data = defaultdict(list)
      self.__unigram_data['Original_para'] = self.__dataframe['paragraph']
      self.__unigram_data['Preprocessed_para'] = self.preprocessed
      kw_model = KeyBERT()
      self.unigram_keywords = []
      for index,content in enumerate(self.preprocessed):
        if content:
          key_words = key_model.extract_keywords(content, top_n=5)
          print(index,key_words)
          self.__unigram_data['key_word'].append(key_words[0][0])
          self.__unigram_data['probability'].append(key_words[0][1])
          self.unigram_keywords.append(key_words[0][0])
        else:
          print(index,"Empty value")
          self.__unigram_data['key_word'].append("Nan")
          self.__unigram_data['probability'].append("Nan")
          self.unigram_keywords.append("Nan")

      self.__unigramdf = pd.DataFrame(self.__unigram_data)

      if save:
        path = "/content/drive/MyDrive/Colab Notebooks/unigram_keywords.csv"
        self.__unigramdf.to_csv(path)
        print("CSV file has been successfully saved.")
      
      return self.__unigramdf, self.unigram_keywords

    
  def ngram_key_word_extraction(self,n_grams = 2, save = False):

      """ 
      This method generates ngrams of keywords.
      n_grams ----> Number of keywprds to extract
                    Ex: n_grams = 2--- "Industrial Technology"
                    Ex: n_grams = 3--- "Industrial Technology Standards"
                    Must be 2 or 3
      save -------> Bool, if True, dataframe will be saved to the drive with ngram
                    keywords and their probability
      """

      assert type(n_grams) == int, "n_gram must be an integer"
      assert (n_grams > 1) and (n_grams < 4), "n_gram must be greater than 1 less than 4"

      key_model = KeyBERT()
      self.__ngram_data = defaultdict(list)
      self.__ngram_data['Original_para'] = self.__dataframe['paragraph']
      self.__ngram_data['Preprocessed_para'] = self.preprocessed
      self.ngram_keywords = []
      for index,content in enumerate(self.preprocessed):
        if content:
          list_content = content.split()
          if (len(list_content) > n_grams):
            key_words1 = key_model.extract_keywords(content, keyphrase_ngram_range=(n_grams,n_grams), top_n = 5)
            print(index,key_words1)
            self.__ngram_data['key_word'].append(key_words1[0][0])
            self.__ngram_data['probability'].append(key_words1[0][1])
            self.ngram_keywords.append(key_words1[0][0])
          else:
            print(index,"Empty String")
            self.__ngram_data['key_word'].append("Nan")
            self.__ngram_data['probability'].append("Nan")
            self.ngram_keywords.append("Nan")

        else:
          print(index,"Empty String")
          self.__ngram_data['key_word'].append("Nan")
          self.__ngram_data['probability'].append("Nan")
          self.ngram_keywords.append("Nan")


      self.__ngramdf = pd.DataFrame(self.__ngram_data)

      if save:
        path = "/content/drive/MyDrive/Colab Notebooks/ngram_keywords.csv"
        self.__ngramdf.to_csv(path)
        print("CSV file has been successfully saved.")

      
      return self.__ngramdf, self.ngram_keywords

  def check_relevance(self,specific_keyword,threshold ,save = False):

    """
    This method check whther the the paragraphs are relevant with the givcen specific word or sentence.
    Symmetric and asymmetric semantic search is used here to get the probabilities and 

    specific_keyword ---> string: Can bea word or a small sentence
    threshold ----------> int or float: Threshold value where the relevancy is checked.
    
    """

    assert (type(threshold == int)) or (type(threshold) == float), "Threshold must be an integer or float"
    self.__keyword_search = defaultdict(list)
    self.__keyword_search['orginal paras'] = self.__dataframe['paragraph']
    self.__keyword_search['preprocessed paras'] = self.preprocessed
    query_embedding_sym = symmetric_model.encode(specific_keyword, convert_to_tensor=True)
    query_embedding_asym = asymmetric_model.encode(specific_keyword, convert_to_tensor=True)
    for index,content in enumerate(self.preprocessed):
      len_content = len(content.split())
      if (len_content > 5):
        print(index, "Asymmetric semantic search")
        corpus_embedding = asymmetric_model.encode(content, convert_to_tensor = True)
        similarity_score = util.semantic_search(query_embedding_asym, corpus_embedding, top_k = 4)
        if (similarity_score[0][0]['score'] > threshold):
          self.__keyword_search['Relevancy'].append("Relevant")
          self.__keyword_search['Probability'].append(similarity_score[0][0]['score'])
        else:
          self.__keyword_search['Relevancy'].append("Not Relevant")
          self.__keyword_search['Probability'].append(similarity_score[0][0]['score'])
      else:
        print(index,"Symmetric semantice search")
        corpus_embedding1 = symmetric_model.encode(content, convert_to_tensor = True)
        similarity_score1 = util.semantic_search(query_embedding_sym, corpus_embedding1, top_k = 4)
        if (similarity_score1[0][0]['score'] > threshold):
          self.__keyword_search['Relevancy'].append("Relevant")
          self.__keyword_search['Probability'].append(similarity_score1[0][0]['score'])
        else:
          self.__keyword_search['Relevancy'].append("Not Relevant")
          self.__keyword_search['Probability'].append(similarity_score1[0][0]['score'])

    self.__keyword_dataframe = pd.DataFrame(self.__keyword_search)

    if save:
      path = "/content/drive/MyDrive/Colab Notebooks/key_wordsearch.csv"
      self.__keyword_dataframe.to_csv(path)
      print("CSV file has been successfully saved.")


    return self.__keyword_dataframe

In [65]:
df1 = data.iloc[:1000,:]

In [66]:
p2 = para_relevancy(df1)


 Preprocessing has been completed


In [56]:
df,list1 = p2.unigram_key_word_extraction(save=True)

0 [('organisation', 0.4662), ('chain', 0.4375), ('jewellery', 0.4148), ('practices', 0.4012), ('management', 0.3959)]
1 [('stakeholders', 0.4622), ('stakeholder', 0.411), ('partnerships', 0.3659), ('sustainability', 0.3329), ('organisation', 0.3205)]
2 [('jewelers', 0.5093), ('diamond', 0.4632), ('company', 0.4522), ('founded', 0.4509), ('diamonds', 0.4491)]
3 [('organisations', 0.4939), ('practices', 0.492), ('membership', 0.3902), ('members', 0.3486), ('companies', 0.3374)]
4 [('actions', 0.4753), ('partnership', 0.4367), ('support', 0.4029), ('responsible', 0.4017), ('organisation', 0.3946)]
5 [('stakeholders', 0.483), ('organisation', 0.4635), ('jewellery', 0.4445), ('value', 0.4279), ('sustainability', 0.3738)]
6 [('sustainable', 0.4197), ('rjc', 0.4178), ('resilience', 0.4143), ('development', 0.305), ('impact', 0.2894)]
7 [('organisations', 0.464), ('ungc', 0.3943), ('sector', 0.3731), ('corporate', 0.3587), ('sustainability', 0.3508)]
8 [('sustainable', 0.4522), ('agenda', 0.36

In [57]:
df.head()

,Original_para,Preprocessed_para,key_word,probability
0,chain that promotes trust in the global jewell...,chain promotes trust global jewellery Responsi...,organisation,0.4662
1,Jewellery has always played a special role in ...,Jewellery played special role people lives car...,stakeholders,0.4622
2,It was founded in 2005 by 14 member organisati...,founded member organisations included ABN AMRO...,jewelers,0.5093
3,"Today our membership has grown to more than 1,...",Today membership grown organisations committed...,organisations,0.4939
4,Our actions support the industry to be even mo...,actions support industry responsible strengthe...,actions,0.4753


In [58]:
df1, list2 = p2.ngram_key_word_extraction(save = True)

0 [('organisation jewellery', 0.6192), ('jewellery council', 0.5978), ('chain promotes', 0.5736), ('jewellery responsible', 0.5448), ('responsible jewellery', 0.542)]
1 [('stakeholders increasingly', 0.4986), ('making stakeholders', 0.4712), ('strengthen stakeholder', 0.4654), ('stakeholders rjc', 0.4643), ('impact stakeholders', 0.4599)]
2 [('jewellery confederation', 0.5599), ('diarough jewelers', 0.5384), ('zale corporation', 0.5318), ('diamonds cartier', 0.5138), ('jewelers america', 0.5006)]
3 [('business practices', 0.5607), ('code practices', 0.5429), ('organisations committed', 0.5305), ('practices implementation', 0.5298), ('grown organisations', 0.5159)]
4 [('actions support', 0.6292), ('industry responsible', 0.5764), ('support industry', 0.5564), ('responsible business', 0.5259), ('responsible strengthening', 0.4649)]
5 [('value organisation', 0.6426), ('value chain', 0.497), ('chain jewellery', 0.4955), ('member organisation', 0.4866), ('organisation members', 0.4666)]
6 [

In [59]:
df1.head()

,Original_para,Preprocessed_para,key_word,probability
0,chain that promotes trust in the global jewell...,chain promotes trust global jewellery Responsi...,organisation jewellery,0.6192
1,Jewellery has always played a special role in ...,Jewellery played special role people lives car...,stakeholders increasingly,0.4986
2,It was founded in 2005 by 14 member organisati...,founded member organisations included ABN AMRO...,jewellery confederation,0.5599
3,"Today our membership has grown to more than 1,...",Today membership grown organisations committed...,business practices,0.5607
4,Our actions support the industry to be even mo...,actions support industry responsible strengthe...,actions support,0.6292


In [67]:
df2 = p2.check_relevance("sustainability", 0.3, save=True)

0 Asymmetric semantice search
1 Asymmetric semantice search
2 Asymmetric semantice search
3 Asymmetric semantice search
4 Asymmetric semantice search
5 Asymmetric semantice search
6 Asymmetric semantice search
7 Asymmetric semantice search
8 Asymmetric semantice search
9 Asymmetric semantice search
10 Asymmetric semantice search
11 Asymmetric semantice search
12 Asymmetric semantice search
13 Asymmetric semantice search
14 Asymmetric semantice search
15 Asymmetric semantice search
16 Asymmetric semantice search
17 Asymmetric semantice search
18 Asymmetric semantice search
19 Asymmetric semantice search
20 Asymmetric semantice search
21 Asymmetric semantice search
22 Asymmetric semantice search
23 Asymmetric semantice search
24 Asymmetric semantice search
25 Asymmetric semantice search
26 Asymmetric semantice search
27 Asymmetric semantice search
28 Asymmetric semantice search
29 Asymmetric semantice search
30 Asymmetric semantice search
31 Asymmetric semantice search
32 Asymmetric sema

In [68]:
df2.head()

,orginal paras,preprocessed paras,Relevancy,Probability
0,chain that promotes trust in the global jewell...,chain promotes trust global jewellery Responsi...,Not Relevant,0.231637
1,Jewellery has always played a special role in ...,Jewellery played special role people lives car...,Not Relevant,0.185249
2,It was founded in 2005 by 14 member organisati...,founded member organisations included ABN AMRO...,Not Relevant,0.091612
3,"Today our membership has grown to more than 1,...",Today membership grown organisations committed...,Not Relevant,0.012436
4,Our actions support the industry to be even mo...,actions support industry responsible strengthe...,Not Relevant,0.152047


In [73]:
p3 = para_relevancy(df1)


 Preprocessing has been completed


In [77]:
d1 = p3.check_relevance("energy saving", 0.2)

0 Asymmetric semantic search
1 Asymmetric semantic search
2 Asymmetric semantic search
3 Asymmetric semantic search
4 Asymmetric semantic search
5 Asymmetric semantic search
6 Asymmetric semantic search
7 Asymmetric semantic search
8 Asymmetric semantic search
9 Asymmetric semantic search
10 Asymmetric semantic search
11 Asymmetric semantic search
12 Asymmetric semantic search
13 Asymmetric semantic search
14 Asymmetric semantic search
15 Asymmetric semantic search
16 Asymmetric semantic search
17 Asymmetric semantic search
18 Asymmetric semantic search
19 Asymmetric semantic search
20 Asymmetric semantic search
21 Asymmetric semantic search
22 Asymmetric semantic search
23 Asymmetric semantic search
24 Asymmetric semantic search
25 Asymmetric semantic search
26 Asymmetric semantic search
27 Asymmetric semantic search
28 Asymmetric semantic search
29 Asymmetric semantic search
30 Asymmetric semantic search
31 Asymmetric semantic search
32 Asymmetric semantic search
33 Asymmetric semant

In [78]:
d1.head()

,orginal paras,preprocessed paras,Relevancy,Probability
0,chain that promotes trust in the global jewell...,chain promotes trust global jewellery Responsi...,Not Relevant,0.049949
1,Jewellery has always played a special role in ...,Jewellery played special role people lives car...,Not Relevant,0.053529
2,It was founded in 2005 by 14 member organisati...,founded member organisations included ABN AMRO...,Not Relevant,0.012260
3,"Today our membership has grown to more than 1,...",Today membership grown organisations committed...,Not Relevant,-0.054814
4,Our actions support the industry to be even mo...,actions support industry responsible strengthe...,Not Relevant,0.083860


In [79]:
d1['Relevancy'].value_counts()

Not Relevant    957
Relevant         43
Name: Relevancy, dtype: int64